In [6]:
%load_ext autoreload
%autoreload 2

from os import sys
sys.path.insert(0, '..')

import torch
import os
from tqdm.notebook import tqdm
from models.diffusion import *
from datasets.shapenet import *
from torch.utils.data import DataLoader
from models.latent_cond.models import *
from torch.optim.swa_utils import AveragedModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#!g1.1
batch_size = 40
lr = 2e-4
weight_decay = 0
epoch_num = 1000
ema_beta = 0.999
device = 'cuda:0'

In [8]:
#!g1.1
def compute_diffusion_loss(y, x):
    return (y - x).pow(2).mean()

def compute_kl_loss(mean, log_var, current_step, total_steps, gamma=1000):
    kl_loss = (-0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp(), dim=1)).mean()
    # C = min(25 * current_step / total_steps, 25)
    return kl_loss# gamma * (kl_loss - C).abs() kl_loss
    
def compute_disc_loss(real, fake):
    labels = torch.cat((torch.ones_like(real), torch.zeros_like(fake)), dim=0)
    logits = torch.cat((real, fake), dim=0)
    return torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)

def compute_encoder_loss(logits):
    labels = torch.ones_like(logits)
    return torch.nn.functional.binary_cross_entropy_with_logits(logits, labels)

def get_ema_fn(beta):
    def fn(averaged_model_parameter, model_parameter, num_averaged):
        return beta * averaged_model_parameter + (1 - beta) * model_parameter

    return fn

def normalize(x):
    mean = x.mean(dim=2, keepdim=True)
    std = x.std(dim=(1, 2), keepdim=True)
    return (x - mean) / std

In [ ]:
train_dataset = Dataset('../../datasets', 'shapenetcorev2', load_name=True,
                        transform=None, split='trainval', class_choice='airplane')

val_dataset = Dataset('../../datasets', 'shapenetcorev2', load_name=True,
                      split='test', transform=None, class_choice='airplane')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True,
                          num_workers=2)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True,
                        num_workers=2)

In [5]:
from tqdm.notebook import tqdm

def compute_stats(dataset):
    points = []
    
    for item in tqdm(dataset):
        points.append(item)
        
    points = torch.cat(points, dim=0)
    
    mean = points.mean(dim=(0, 2)).view(1, 3, 1)
    
    std = points.view(-1).std(dim=0, unbiased=True).view(1, 1, 1)
    
    return mean, std

In [6]:
model = DiffusionModel(NoisePredictor(3, residual=True),
                       1000,
                       time_embedding_dim=3).to(device)

avg_model = AveragedModel(model, avg_fn=get_ema_fn(ema_beta))

optimizer = torch.optim.Adam(model.parameters(),
                             lr=lr,
                             weight_decay=weight_decay,
                             betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-4, T_max=len(train_loader) * epoch_num)

In [7]:
sum(p.numel() for p in model.parameters())

9475358

In [8]:
#!g1.1
running_diff_loss = 0
running_kl_loss = 0

cur_iter = 0
cur_step = 0
total_steps = len(train_loader) * epoch_num

for epoch in range(epoch_num):
    bar = tqdm(train_loader)
    model.train()
    # discriminator.train()
    
    for x in bar:
        # diffusion
        optimizer.zero_grad()
        # x = ((x - mean_global) / std_global).to(device)
        x = x.to(device)
        e, (et, ctx, (z, mean, log_var)) = model(x)
    
        kl_loss = compute_kl_loss(mean, log_var, cur_step, total_steps)
        diff_loss = compute_diffusion_loss(e, et)
        
        loss = diff_loss + kl_loss
        loss.backward()
        
        optimizer.step()
        scheduler.step()      

        avg_model.update_parameters(model)

        running_diff_loss += diff_loss.item()
        running_kl_loss += kl_loss.item()
        
        cur_iter += 1
        cur_step += 1
        
        bar.set_postfix({
            'Epoch': epoch,
            'Diff Loss': running_diff_loss / cur_iter,
            'KL Loss': running_kl_loss / cur_iter,
        })
    
    # run['train_loss'].log(running_loss / cur_iter)
    
    model.eval()
    val_loss = 0
    bar = tqdm(enumerate(val_loader, start=1), total=len(val_loader))

    with torch.no_grad():
        for i, batch in bar:
            # batch = normalize(batch.to(device))
            e, (et, ctx, (z, mean, log_var)) = model(batch.to(device))
            loss = compute_diffusion_loss(e, et)
            val_loss += loss.item()

            bar.set_postfix({
                'Loss': val_loss / i
            })
    
    # run['val_loss'].log(val_loss / len(val_loader))
    
    if (epoch + 1) % 20 == 0:
        print('Sampling...')
        samples = model.sample(track_trajectory=False, batch_size=5)
        torch.save(samples, f'samples/{epoch + 1}.pt')
            
    
    if (epoch + 1) % 100 == 0:
        state = {
            'model': model.state_dict(),
            'ema': avg_model.module.state_dict()
        }
        torch.save(state, f'model_{epoch}.pt')
        
    running_diff_loss = 0
    running_kl_loss = 0

    cur_iter = 0


  0%|          | 0/1000 [00:00<?, ?it/s]


Sampling...


100%|██████████| 1000/1000 [04:29<00:00,  3.72it/s]


  0%|          | 0/1000 [00:00<?, ?it/s]


Sampling...


...

100%|██████████| 1000/1000 [06:20<00:00,  2.63it/s]


  0%|          | 0/1000 [00:00<?, ?it/s]


Sampling...


 64%|██████▍   | 638/1000 [04:04<02:18,  2.61it/s]


KeyboardInterrupt: 